In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2

In [2]:
from src.models.modules import *
from src.models.loss import L1_clamp_loss
from dataclasses import dataclass
import torch

@dataclass
class SDFTransformerConfig:
    dim_context: int
    dim_input: int
    num_outputs: int
    dim_output: int
    delta: float = 0.1
    num_inds: int = 32
    dim_hidden: int = 128
    num_heads: int = 8
    ln: bool = False

class SDFTransformer(nn.Module):
    def __init__(self, config: SDFTransformerConfig):
        super(SDFTransformer, self).__init__()
        self.config = config
        self.enc = nn.Sequential(
            ISAB(config.dim_context, config.dim_hidden, config.num_heads, config.num_inds, ln=config.ln),
            ISAB(config.dim_hidden, config.dim_hidden, config.num_heads, config.num_inds, ln=config.ln)
        )
        self.input_proj = nn.Sequential(nn.Linear(config.dim_input, config.dim_hidden), nn.ReLU())
        self.cross = MAB(config.dim_hidden, config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln)
        self.dec = nn.Sequential(
            PMA(config.dim_hidden, config.num_heads, config.num_outputs, ln=config.ln),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
        )
        self.regr = nn.Sequential(
            nn.ReLU(),
            nn.Linear(config.dim_hidden, config.dim_output),
            nn.Tanh()
        )

    def forward(self, context: torch.Tensor, x: torch.Tensor, labels: torch.Tensor = None):
        y = self.enc(context)           # [batch_size, context_size, dim_hidden]
        x = self.input_proj(x)          # [batch_size, num_outputs, dim_hidden]
        x = x.repeat(1, y.shape[1], 1)  # [batch_size, context_size, dim_hidden]
        y = self.cross(x, y)            # [batch_size, context_size, dim_hidden]
        y = self.dec(y)                 # [batch_size, num_outputs, dim_hidden]
        y = self.regr(y)                # [batch_size, num_outputs, dim_output]

        loss = None
        if labels is not None:
            loss = L1_clamp_loss(y, labels, self.config.delta)
        return {'loss': loss, 'logits': y}

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = SDFTransformerConfig(dim_context=4, dim_input=3, num_outputs=1, dim_output=1)
model = SDFTransformer(config).to(device)
print(device)

cuda


In [3]:
from src.models.dataset import LazySampleDataset
from pathlib import Path

project_dir = Path(os.path.abspath('')).resolve().parent
procesed_dir = project_dir / 'data' / 'processed'

train_files = list(procesed_dir.rglob('*_train.hdf5'))
val_files = list(procesed_dir.rglob('*_val.hdf5'))

train_dataset = LazySampleDataset(train_files)
val_dataset = LazySampleDataset(val_files)

In [4]:
from src.data.load_data import get_results_dir
from datetime import datetime

notebook_name = '2024_11_22_more_heads'
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
folder_name = f"{notebook_name}-{current_date}"
result_dir = get_results_dir() / folder_name
result_dir.mkdir(parents=True, exist_ok=True)
print(result_dir)

C:\_prog\vm_shared\attention-sdf\results\2024_11_22_more_heads-2024-11-22-15-07-24


In [5]:
from transformers import Trainer, TrainingArguments

batch_size = 40
training_args = TrainingArguments(
    output_dir=result_dir / "results",
    eval_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=8,
    logging_dir=result_dir / "logs",
    logging_steps=10,
    weight_decay=0.01,
    save_total_limit=3,
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [6]:
trainer.train()
train_dataset.close()
val_dataset.close()

  0%|          | 0/120000 [00:00<?, ?it/s]

{'loss': 0.0269, 'grad_norm': 2.687889814376831, 'learning_rate': 4.999583333333333e-05, 'epoch': 0.0}
{'loss': 0.0115, 'grad_norm': 2.5180647373199463, 'learning_rate': 4.999166666666667e-05, 'epoch': 0.0}
{'loss': 0.0083, 'grad_norm': 1.7175499200820923, 'learning_rate': 4.99875e-05, 'epoch': 0.0}
{'loss': 0.0069, 'grad_norm': 2.182140588760376, 'learning_rate': 4.998333333333334e-05, 'epoch': 0.0}
{'loss': 0.0083, 'grad_norm': 0.3744048774242401, 'learning_rate': 4.997916666666667e-05, 'epoch': 0.0}
{'loss': 0.0094, 'grad_norm': 1.4175785779953003, 'learning_rate': 4.9975e-05, 'epoch': 0.0}
{'loss': 0.0083, 'grad_norm': 2.287304401397705, 'learning_rate': 4.997083333333333e-05, 'epoch': 0.0}
{'loss': 0.0104, 'grad_norm': 2.603260040283203, 'learning_rate': 4.996666666666667e-05, 'epoch': 0.01}
{'loss': 0.0089, 'grad_norm': 1.9657002687454224, 'learning_rate': 4.99625e-05, 'epoch': 0.01}
{'loss': 0.0086, 'grad_norm': 1.6856985092163086, 'learning_rate': 4.995833333333333e-05, 'epoch'

  0%|          | 0/5000 [00:00<?, ?it/s]

{'eval_loss': 0.003109253942966461, 'eval_runtime': 153.4018, 'eval_samples_per_second': 1303.765, 'eval_steps_per_second': 32.594, 'epoch': 1.0}
{'loss': 0.0035, 'grad_norm': 0.697627604007721, 'learning_rate': 4.374583333333334e-05, 'epoch': 1.0}
{'loss': 0.0045, 'grad_norm': 0.6428651213645935, 'learning_rate': 4.374166666666667e-05, 'epoch': 1.0}
{'loss': 0.0036, 'grad_norm': 0.6877717971801758, 'learning_rate': 4.3737500000000006e-05, 'epoch': 1.0}
{'loss': 0.0033, 'grad_norm': 0.8344091176986694, 'learning_rate': 4.373333333333334e-05, 'epoch': 1.0}
{'loss': 0.0026, 'grad_norm': 0.17249363660812378, 'learning_rate': 4.372916666666667e-05, 'epoch': 1.0}
{'loss': 0.0031, 'grad_norm': 0.19551633298397064, 'learning_rate': 4.3725000000000006e-05, 'epoch': 1.0}
{'loss': 0.004, 'grad_norm': 0.7089709043502808, 'learning_rate': 4.372083333333333e-05, 'epoch': 1.0}
{'loss': 0.0027, 'grad_norm': 0.6577184796333313, 'learning_rate': 4.371666666666667e-05, 'epoch': 1.01}
{'loss': 0.0036, 'g

  0%|          | 0/5000 [00:00<?, ?it/s]

...

{'eval_loss': 0.001887280261144042, 'eval_runtime': 186.8993, 'eval_samples_per_second': 1070.095, 'eval_steps_per_second': 26.752, 'epoch': 6.0}
{'loss': 0.0016, 'grad_norm': 0.10652846097946167, 'learning_rate': 1.2495833333333335e-05, 'epoch': 6.0}
{'loss': 0.0029, 'grad_norm': 0.4427286684513092, 'learning_rate': 1.2491666666666668e-05, 'epoch': 6.0}
{'loss': 0.0018, 'grad_norm': 0.021704087033867836, 'learning_rate': 1.24875e-05, 'epoch': 6.0}
{'loss': 0.0026, 'grad_norm': 0.046848248690366745, 'learning_rate': 1.2483333333333335e-05, 'epoch': 6.0}
{'loss': 0.0021, 'grad_norm': 0.7478758096694946, 'learning_rate': 1.2479166666666667e-05, 'epoch': 6.0}
{'loss': 0.0018, 'grad_norm': 0.14579825103282928, 'learning_rate': 1.2475e-05, 'epoch': 6.0}
{'loss': 0.0026, 'grad_norm': 0.15194548666477203, 'learning_rate': 1.2470833333333334e-05, 'epoch': 6.0}
{'loss': 0.0018, 'grad_norm': 0.20786215364933014, 'learning_rate': 1.2466666666666667e-05, 'epoch': 6.01}
{'loss': 0.0016, 'grad_norm'

  0%|          | 0/5000 [00:00<?, ?it/s]

{'eval_loss': 0.0018075663829222322, 'eval_runtime': 155.4724, 'eval_samples_per_second': 1286.402, 'eval_steps_per_second': 32.16, 'epoch': 7.0}
{'loss': 0.0023, 'grad_norm': 0.030681638047099113, 'learning_rate': 6.245833333333334e-06, 'epoch': 7.0}
{'loss': 0.0015, 'grad_norm': 0.3774433732032776, 'learning_rate': 6.241666666666667e-06, 'epoch': 7.0}
{'loss': 0.0018, 'grad_norm': 0.205609530210495, 'learning_rate': 6.2375e-06, 'epoch': 7.0}
{'loss': 0.0018, 'grad_norm': 0.03601944074034691, 'learning_rate': 6.2333333333333335e-06, 'epoch': 7.0}
{'loss': 0.0028, 'grad_norm': 0.27060166001319885, 'learning_rate': 6.229166666666667e-06, 'epoch': 7.0}
{'loss': 0.0022, 'grad_norm': 0.10506592690944672, 'learning_rate': 6.2250000000000005e-06, 'epoch': 7.0}
{'loss': 0.0022, 'grad_norm': 0.3234265148639679, 'learning_rate': 6.220833333333333e-06, 'epoch': 7.0}
{'loss': 0.0014, 'grad_norm': 0.16920217871665955, 'learning_rate': 6.2166666666666676e-06, 'epoch': 7.01}
{'loss': 0.0016, 'grad_n

  0%|          | 0/5000 [00:00<?, ?it/s]

{'eval_loss': 0.0017758599715307355, 'eval_runtime': 168.8164, 'eval_samples_per_second': 1184.719, 'eval_steps_per_second': 29.618, 'epoch': 8.0}
{'train_runtime': 8125.7896, 'train_samples_per_second': 590.712, 'train_steps_per_second': 14.768, 'train_loss': 0.002325299289325873, 'epoch': 8.0}


In [7]:
import json
from dataclasses import asdict

current_date = datetime.now().strftime("%Y-%m-%d")
model_name = f"{current_date}-model"
config_name = f"{current_date}-config.json"
trainer.save_model(result_dir / model_name)

with open(result_dir / config_name, 'w') as f:
    json.dump(asdict(config), f)

In [8]:
from src.visualization.generate_mesh import generate_mesh
from src.data.load_data import get_data_dir

obj_path = get_data_dir() / 'intermediate' / 'bunny' / 'stanford-bunny.obj'
infered_obj_path = result_dir / f"bunny-{current_date}-generalization.obj"
generate_mesh(model, obj_path, infered_obj_path, device, batch_size, resolution=100)

Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]

In [9]:
hdf5_path = get_data_dir() / 'processed' / 'bunny' / 'stanford-bunny_train.hdf5'
infered_obj_path = result_dir / f"{current_date}-train.obj"
generate_mesh(model, hdf5_path, infered_obj_path, device, batch_size, resolution=100)

Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]